In [4]:
import torch

a = torch.load('/mnt/SrvUserDisk/JiangXiaotian/workspace/3DEM/ref/decoded.pt')  # 你可以根据实际情况初始化矩阵 a
b = torch.load('/mnt/SrvUserDisk/JiangXiaotian/workspace/3DEM/ref/gt.pt')  # 你可以根据实际情况初始化矩阵 b
print(a.shape)
c = a - b
d = c ** 2
mean_value = d.sum() / d.numel()

print(mean_value)

torch.Size([2, 361, 720])
tensor(0.0212, device='cuda:0', grad_fn=<DivBackward0>)


In [10]:
import torch

# 加载张量数据
a = torch.load('/mnt/SrvUserDisk/JiangXiaotian/workspace/3DEM/ref/decoded.pt')  # 预测矩阵
b = torch.load('/mnt/SrvUserDisk/JiangXiaotian/workspace/3DEM/ref/gt.pt')  # 真实值矩阵
print(a.shape)
# 检查形状是否匹配
if a.shape != b.shape:
    raise ValueError("预测值和真实值的形状不匹配！")

# 差值
c = a - b

# MSE (Mean Squared Error)
mse = (c ** 2).mean()

# NMSE (Normalized Mean Squared Error)
# 通常定义为 MSE / (真实值的平方均值)
# nmse = mse / (b ** 2).mean()
nmse = mse / b.var()

rmse = torch.sqrt(mse)

# L1 Loss (Mean Absolute Error)
l1_loss = c.abs().mean()

# 百分比误差 (Percentage Error, 按元素)
# 防止真实值中有零，导致除零错误
percentage_error = (c / (b + 1e-8)).abs().mean() * 100  # 结果以百分比形式表示

# 输出结果
print(f"MSE: {mse.item():.4f}")
print(f"NMSE: {nmse.item():.4f}")
print(f"RMSE: {rmse.item():.4f}")
print(f"L1 Loss: {l1_loss.item():.4f}")
print(f"Percentage Error: {percentage_error.item():.2f}%")

torch.Size([2, 361, 720])
MSE: 0.0212
NMSE: 0.0109
RMSE: 0.1457
L1 Loss: 0.1164
Percentage Error: 38.98%


In [13]:
import torch

# 假设 a 和 b 的 shape 为 [batch_size, height, width]
# 加载数据
a = torch.load('/mnt/SrvUserDisk/JiangXiaotian/workspace/3DEM/ref/decoded.pt')  # 预测值
b = torch.load('/mnt/SrvUserDisk/JiangXiaotian/workspace/3DEM/ref/gt.pt')  # 真实值

# 检查形状是否匹配
if a.shape != b.shape:
    raise ValueError("预测值和真实值的形状不匹配！")

# 差值
c = a - b

# MSE (Mean Squared Error) for each image
mse_per_image = (c ** 2).mean(dim=(1, 2))  # 按单张图像计算 MSE
mse_mean = mse_per_image.mean()  # 对所有图像的 MSE 求均值

# NMSE (Normalized Mean Squared Error) for each image
# 使用真实值的方差归一化
nmse_per_image = mse_per_image / b.var(dim=(1, 2), unbiased=False)  # 无偏估计
nmse_mean = nmse_per_image.mean()  # 对所有图像的 NMSE 求均值

# RMSE (Root Mean Squared Error) for each image
rmse_per_image = torch.sqrt(mse_per_image)  # 每张图像的 RMSE
rmse_mean = rmse_per_image.mean()  # 所有图像的 RMSE 均值

# L1 Loss (Mean Absolute Error) for each image
l1_loss_per_image = c.abs().mean(dim=(1, 2))  # 每张图像的 L1 Loss
l1_loss_mean = l1_loss_per_image.mean()  # 所有图像的 L1 Loss 均值

# Percentage Error (按元素百分比误差)
# 防止真实值为 0 导致除零问题，添加一个很小的值 1e-8
percentage_error_per_image = (c / (b + 1e-8)).abs().mean(dim=(1, 2)) * 100  # 每张图像的百分比误差
percentage_error_mean = percentage_error_per_image.mean()  # 所有图像的百分比误差均值

# 输出结果
print(f"MSE (per image): {mse_per_image}")
print(f"MSE Mean: {mse_mean.item():.4f}")
print(f"NMSE (per image): {nmse_per_image}")
print(f"NMSE Mean: {nmse_mean.item():.4f}")
print(f"RMSE (per image): {rmse_per_image}")
print(f"RMSE Mean: {rmse_mean.item():.4f}")
print(f"L1 Loss (per image): {l1_loss_per_image}")
print(f"L1 Loss Mean: {l1_loss_mean.item():.4f}")
print(f"Percentage Error (per image): {percentage_error_per_image}")
print(f"Percentage Error Mean: {percentage_error_mean.item():.2f}%")

MSE (per image): tensor([0.0224, 0.0200], device='cuda:0', grad_fn=<MeanBackward1>)
MSE Mean: 0.0212
NMSE (per image): tensor([0.9794, 0.0197], device='cuda:0', grad_fn=<DivBackward0>)
NMSE Mean: 0.4995
RMSE (per image): tensor([0.1498, 0.1414], device='cuda:0', grad_fn=<SqrtBackward0>)
RMSE Mean: 0.1456
L1 Loss (per image): tensor([0.1235, 0.1092], device='cuda:0', grad_fn=<MeanBackward1>)
L1 Loss Mean: 0.1164
Percentage Error (per image): tensor([70.9961,  6.9705], device='cuda:0', grad_fn=<MulBackward0>)
Percentage Error Mean: 38.98%
